## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [35]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [36]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [37]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Ravi and I am a AI Engineer")])

AIMessage(content="Hello Ravi, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or emotions, but I'm always eager to learn and chat. What can I do for you today? Are you working on any interesting projects as an AI Engineer?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 21, 'total_tokens': 81, 'completion_time': 0.109090909, 'prompt_time': 0.000153759, 'queue_time': 0.020256459, 'total_time': 0.109244668}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4bca71d-a5a0-47da-9fcb-444a64968f6d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 60, 'total_tokens': 81})

In [38]:
# passing history to check if it remembers or not

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Ravi and I am AI Engineer"),
        AIMessage(content="Hi Ravi, it's nice to meet you! \n\nAs an AI, I don't have personal experiences like being an engineer, but I'm always eager to learn more about what you do.  \n\nWhat kind of AI engineering are you involved in?  Do you have any interesting projects you're working on?\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Ravi and that you are an AI Engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm all ears!  👂  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 111, 'total_tokens': 161, 'completion_time': 0.090909091, 'prompt_time': 0.005407815, 'queue_time': 0.07429688300000001, 'total_time': 0.096316906}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4483fad0-7604-4585-a5c5-a1cc07c7b518-0', usage_metadata={'input_tokens': 111, 'output_tokens': 50, 'total_tokens': 161})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [39]:
# !pip install langchain_community

In [40]:
# utility for storing and managing the history of chat messages in a conversation
from langchain_community.chat_message_histories import ChatMessageHistory

# base class for implementing your own custom chat history. 
# If you need to store messages in a database, a file, or a custom data structure
# instead of the default storage, you can extend BaseChatMessageHistory to create a custom implementation
from langchain_core.chat_history import BaseChatMessageHistory

# log the input and output messages of a runnable (like a function or operation) 
# in the context of a chat-based interaction
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    """
      This is to make sure every user have unique chat interaction. session_id is string and
      it inherit BaseChatHistory.
    """
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [41]:
# creating chat1 as first session id
config = {"configurable":{"session_id":"chat1"}}

In [42]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Ravi and I am AI Engineer")],
    config=config
)

In [43]:
response.content

"Hi Ravi, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn and chat. What can I do for you today? Are you working on any interesting projects?\n"

In [44]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Ravi.  \n\nI remember that from our earlier conversation. 😊  How can I help you further? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 90, 'total_tokens': 118, 'completion_time': 0.050909091, 'prompt_time': 0.002258843, 'queue_time': 0.021005487, 'total_time': 0.053167934}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d5bae4b-d206-4176-a37b-1154492c8310-0', usage_metadata={'input_tokens': 90, 'output_tokens': 28, 'total_tokens': 118})

In [45]:
## change the config-->session id

config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it! 😊\n"

In [46]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Hi John! Nice to meet you.  \n\nIs there anything I can help you with today?\n'

In [47]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  😊 \n\nI remember!  What can I do for you, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [48]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [49]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Ravi")]})

AIMessage(content="Hello Ravi, it's nice to meet you!  \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 31, 'total_tokens': 65, 'completion_time': 0.061818182, 'prompt_time': 0.000343629, 'queue_time': 0.020479987999999998, 'total_time': 0.062161811}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b4c9cae6-0ce1-4ab4-b0bc-92c4b516a4ab-0', usage_metadata={'input_tokens': 31, 'output_tokens': 34, 'total_tokens': 65})

In [50]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [51]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Ravi")],
    config=config
)

response

AIMessage(content="Hi Ravi, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away, and I'll do my best to answer them! 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.000305469, 'queue_time': 0.019393339000000002, 'total_time': 0.082123651}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3aa7e27f-0704-4504-96fd-b3ada082fe1a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [52]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ravi, you told me! 😊  \n\nIs there anything else I can help you with?\n'

In [53]:
## Add more complexity

# Define the ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        # System message to set the behavior of the assistant
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        
        # Placeholder for dynamic conversation history
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [54]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Ravi")],"language":"Hindi"})
response.content

'नमस्ते रवि! 👋  \n\nमुझे आपसे बातचीत करने में खुशी हो रही है।  \n\nआप मुझसे क्या पूछना चाहते हैं? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [55]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [56]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Ravi")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते रवि! 👋  मैं आपकी मदद करने के लिए यहाँ हूँ।  आप क्या जानना चाहेंगे? 😊\n'

In [57]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [58]:
response.content

'आपका नाम रवि है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [59]:
from langchain_core.messages import SystemMessage,trim_messages

# Define the trimming configuration
trimmer = trim_messages(
    max_tokens=45,                  # Limit the total tokens to 45
    strategy="last",                # Trim the oldest messages first
    token_counter=model,            # Use the specified tokenization model to count tokens
    include_system=True,            # Include the SystemMessage in the trimming
    allow_partial=False,            # Do not allow partial trimming of individual messages
    start_on="human"                # Start trimming from user (human) messages
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [60]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# Define the pipeline
chain = (
    RunnablePassthrough
    .assign(
        messages=itemgetter("messages") | trimmer  # Extract 'messages' and trim them using `trimmer`
    )
    | prompt  # Pass the processed data into the prompt
    | model   # Pass the prompt's output to the model for final processing
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [61]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2" \n\n\nLet me know if you\'d like to try another one!\n'

In [62]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

# session id is different, so it is not giving answer
config={"configurable":{"session_id":"chat5"}}

In [63]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you. So I don't know your name. 😊\n\nWould you like to tell me?\n"

In [64]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"